In [ ]:
!pip install openai --quiet

In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/글평가'

!ls {file_path}

 NanumGothic.ttf	       '단어분류.ipynb의 사본'	 입력텍스트실험.ipynb
 poems_by_poet.gsheet	        버트.ipynb		 차다_2023_news_sentences.csv
 v1_classified_sentences.csv    시험.ipynb		'차다_2023_talk_sentences.csv의 사본'
 v1_classified_sentences.xlsx   실험.ipynb		 차다_통합.csv
 단어분류.ipynb		        싸다_통합.csv		 타다_통합.csv


In [ ]:
import json
import csv
import pandas as pd
from openai import OpenAI

# api key 입력
client = OpenAI(api_key="자신의 키를 입력해주세요.")

In [ ]:
from pydantic import BaseModel

class Categorize(BaseModel):
    category: str

In [ ]:
def classify_word_meaning(sentence):  # <-- 매개변수 이름 변경

    # 프롬프트 자체의 상수를 맨 위에 정의해놔도 괜찮을 듯?
    # 프롬프트를 고쳐야 하긴 할 듯
    # 찬밥, 찬바람 / 장갑, 팔찌, 목걸이, 완장, 전자발찌 / 고뇌에 찬, 야심에 찬 / 찰옥수수, 찰순대
    # 가볍게 정의하는 방법 vs 사전적 정의 (뭐가 더 좋을까?)

    prompt = f"""
    주어지는 문장에는 용언 '차다'의 활용형 어간이 대괄호로 하이라이트 되어 있습니다. (ex. [차], [찬], [찰], [찹], [참], [찼] 등)
    대괄호로 하이라이트 된 어간이 다음 4개의 분류(c1, c2, c3, c4) 중 어떤 의미에 가까운지 판단하세요.

    - c1 : 일정한 공간에 사람, 사물, 냄새 따위가 가득하게 되다.
    - c2 : 감정이나 기운 따위가 가득하게 되다.
    - c3 : 어떤 대상이 흡족하게 마음에 들다.
    - c4 : 	어떤 높이나 한도에 이르는 상태가 되다.
    - etc : 위 뜻으로 분류되지 않는 것.

    <실전>
    {sentence} :
    """

    try:
        response = client.beta.chat.completions.parse(
            model="gpt-4o-mini",
            temperature=0,
            messages=[
                {"role": "system", "content": "너는 단어의 뜻을 분류해야 해. 잘 고민하고 느낌을 이해하며 분류해봐."},
                {"role": "user", "content": prompt}
            ],
            response_format=Categorize,
        )

        if response.choices and response.choices[0].message.parsed:
            return response.choices[0].message.parsed.category
        else:
            return "분류 불가"  # 응답 구조가 올바르지 않은 경우
    except Exception as e:
        print(f"카테고리 분류 중 오류 발생: {e}")
        return "오류 발생"  # 오류 발생 시

In [ ]:
# csv 파일 업로드
df = pd.read_csv(file_path + '/v1_classified_sentences.csv')
sampled_df = df.sample(n=50, random_state=50)

# 일부만

In [ ]:
# 일부만

from tqdm import tqdm
import concurrent.futures

tqdm.pandas()

# 병렬 처리 함수
# max_workers 는 4 이상은 바름. 2는 느리지만 안정적.

def classify_parallel(sentences, max_workers=4):
    results = []
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        # map()을 사용하면 입력된 순서대로 결과가 정렬됨
        results = list(tqdm(executor.map(classify_word_meaning, sentences), total=len(sentences)))
    return results

In [ ]:
# 병렬 처리 적용 (입력 순서 유지)
sampled_df["Polysemy"] = classify_parallel(sampled_df["Sentence"])

# CSV 파일로 저장
sampled_df.to_csv("sampled_classified_sentences.csv", index=False)
print("분류된 문장이 'sampled_classified_sentences.csv' 파일로 저장되었습니다.")

100%|██████████| 50/50 [00:09<00:00,  5.17it/s]

분류된 문장이 'classified_sentences.csv' 파일로 저장되었습니다.


# 전체

In [ ]:
# 일부만

from tqdm import tqdm
import concurrent.futures

tqdm.pandas()

# 병렬 처리 함수
# max_workers 는 4 이상은 바름. 2는 느리지만 안정적.

def classify_parallel(sentences, max_workers=4):
    results = []
    with concurrent.futures.ProcessPoolExecutor(max_workers=max_workers) as executor:
        # map()을 사용하면 입력된 순서대로 결과가 정렬됨
        results = list(tqdm(executor.map(classify_word_meaning, sentences), total=len(sentences)))
    return results

In [ ]:
# 전체

df["Polysemy"] = classify_parallel(df["Sentence"])

# CSV 파일로 저장
df.to_csv("classified_sentences.csv", index=False)
print("분류된 문장이 'classified_sentences.csv' 파일로 저장되었습니다.")

100%|██████████| 997/997 [02:37<00:00,  6.33it/s]

분류된 문장이 'classified_sentences.csv' 파일로 저장되었습니다.


In [ ]:
# classified_sentences.csv 를 xlsx로 변경

classified_df = pd.read_csv("classified_sentences.csv")
classified_df.to_excel("classified_sentences.xlsx", index=False)